# Setup

In [1]:
from manifold_twins import ManifoldTwinsAnalysis
from matplotlib import pyplot as plt
import numpy as np
from idrtools import math
from utils import frac_to_mag

In [2]:
%matplotlib widget 

In [3]:
a = ManifoldTwinsAnalysis()
a.settings['figure_directory'] = './figures_generation/'
# a.settings['differential_evolution_use_salt_x1'] = True
a.run_analysis()

Loading dataset...
    IDR:          BLACKSTON
    Phase range: [-5.0, 5.0] days
    Bin velocity: 1000.0


100%|██████████| 415/415 [00:17<00:00, 24.34it/s]


Estimating the spectra at maximum light...
    Loaded cached stan model
    Using saved stan result
Reading between the lines...
    Loaded cached stan model
    Using saved stan result
Building masks...
    Masking 30/203 targets whose uncertainty power is 
    more than 0.100 of the intrinsic power.
Generating the manifold learning embedding...
Calculating spectral indicators...
Fitting RBTL GP to magnitude residuals...
GP magnitude residuals fit:
    Fit result:           b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
    intrinsic_dispersion      0.072 ± 0.012
    gp_kernel_amplitude       0.110 ± 0.042
    gp_length_scale           3.471 ± 2.467
    offset                    -0.029 ± 0.071
    covariate_slope_0         -0.007 ± 0.070
    Fit NMAD                  0.071 mag
    Fit std                   0.098 mag
Calculating SALT2 magnitude residuals...
SALT2 magnitude residuals fit: 
    ref_mag: -10.448
    alpha:   0.142
    beta:    2.677
    σ_int:   0.138
    RMS:     0.1

# Estimating the spectra at maximum light

## Examples of maximum light models

In [4]:
def get_time_evolution_data(idx):
    fluxes = a.flux[a.target_map == idx]
    phases = a.salt_phases[a.target_map == idx]
    model = a.differential_evolution_result['maximum_flux'][idx]
    model_err = a.differential_evolution_result['maximum_fluxerr'][idx]
    gray_offsets = a.differential_evolution_result['gray_offsets'][a.target_map == idx]
    model_diffs = a.differential_evolution_result['model_diffs'][a.target_map == idx]
    
    return fluxes, phases, model, model_err, gray_offsets, model_diffs

def plot_time_evolution_model(idx, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
    fluxes, phases, model, model_err, gray_offsets, model_diffs = get_time_evolution_data(idx)

    for flux, phase in zip(fluxes, phases):
        a.plot_flux(ax, flux, label='Data (%.2f days)' % phase)
    a.plot_flux(ax, model, model_err, c='k', ls='--', label='Model (0 days)')
    ax.legend()
    ax.set_title(a.targets[idx])
    
def plot_time_evolution_difference(idx, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
    fluxes, phases, model, model_err, gray_offsets, model_diffs = get_time_evolution_data(idx)

    phase_slope = a.differential_evolution_result['phase_slope']
    phase_quadratic = a.differential_evolution_result['phase_quadratic']
    gray_offsets = a.differential_evolution_result['gray_offsets'][a.target_map == idx]
    model_diffs = a.differential_evolution_result['model_diffs'][a.target_map == idx]

    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(fluxes, phases, gray_offsets, model_diffs)):
        ax.plot(a.wave, -2.5*np.log10(flux / model), label='Data (%.2f days)' % phase, c='C%d' % i)
    for i, (flux, phase, gray_offset, model_diff) in enumerate(zip(fluxes, phases, gray_offsets, model_diffs)):
        ax.plot(a.wave, model_diff, label='Model (%.2f days)' % phase, c='C%d' % i, ls='--')
    ax.legend(ncol=2, loc=1)
    ax.set_title(a.targets[idx])
    ax.set_xlabel(a.settings['spectrum_plot_xlabel'])
    ax.set_ylabel('Difference from\nmaximum light (mag)')

def plot_time_evolution_residuals(idx, ax=None):
    if ax is None:
        fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
    fluxes, phases, model, model_err, gray_offsets, model_diffs = get_time_evolution_data(idx)
    
    # Plot the interpolation residuals
    for i, (flux, phase, model_diff) in enumerate(zip(fluxes, phases, model_diffs)):
        ax.plot(a.wave, -2.5*np.log10(flux / model) - model_diff, label='Residuals (%.2f days)' % phase, c='C%d' % i)
    ax.legend()
    ax.set_title(a.targets[idx])
    ax.set_xlabel(a.settings['spectrum_plot_xlabel'])
    ax.set_ylabel('Interpolation residuals (mag)')
    
def plot_same_night(idx):
    plot_time_evolution_model(idx)
    plot_time_evolution_difference(idx)
    plot_time_evolution_residuals(idx)

from ipywidgets import interact
interact(plot_same_night, idx=(0, len(a.targets)-1))

interactive(children=(IntSlider(value=101, description='idx', max=202), Output()), _dom_classes=('widget-inter…

<function __main__.plot_same_night(idx)>

In [5]:
plot_targets = ['PTF13ayw', 'SN2004gc']

figsize = a.settings['spectrum_plot_figsize_double']
fig, axes = plt.subplots(2, 1, figsize=figsize, sharex=True)

for ax, plot_target in zip(axes, plot_targets):
    target_names = np.array([i.name for i in a.targets])
    plot_idx = np.where(target_names == plot_target)[0][0]

    plot_time_evolution_model(plot_idx, ax)

for ax in axes[:-1]:
    ax.set_xlabel(None)

a.savefig('time_evolution_model.pdf', fig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Differential time evolution model

In [6]:
cmap = a.settings['colormap']

phase_slope = a.differential_evolution_result['phase_slope']
phase_quadratic = a.differential_evolution_result['phase_quadratic']
phase_slope_x1 = a.differential_evolution_result['phase_slope_x1']
phase_quadratic_x1 = a.differential_evolution_result['phase_quadratic_x1']

def evaluate_phase_difference(phase, x1=0):
    phase_difference = (
        phase_slope * phase
        + phase_quadratic * phase * phase
        + phase_slope_x1 * x1 * phase
        + phase_quadratic_x1 * x1 * phase * phase
    )
    
    return phase_difference

# Look at change in phase for the same x1
max_phase = a.settings['phase_range']
min_phase = -a.settings['phase_range']
num_phases = 10
phases = np.linspace(min_phase, max_phase, num_phases)

fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])

norm = plt.Normalize(vmin=min_phase, vmax=max_phase)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array(phases)

for phase in phases:
    ax.plot(a.wave, evaluate_phase_difference(phase), c=cmap(norm(phase)), zorder=np.abs(phase))

fig.colorbar(sm, ax=ax, orientation='vertical', label='Phase (days)', aspect=40)

ax.set_xlabel(a.settings['spectrum_plot_xlabel'])
ax.set_ylabel('Brightness relative to\n$t_{max,B}$ (mag)')
ax.invert_yaxis()

a.savefig('time_evolution_phase_difference.pdf', fig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
# SALT2 x1 difference plots. This only makes sense if we are including x1 in the
# differential evolution model, so these are blank with the default settings.
# I only make them if x1 was actually used.

def plot_x1_difference(phase):
    # Look at change in phase for the same x1
    min_x1 = -2
    max_x1 = 2
    num_x1s = 10
    x1s = np.linspace(min_x1, max_x1, num_x1s)

    fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
    norm = plt.Normalize(vmin=min_x1, vmax=max_x1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array(x1s)

    for x1 in x1s:
        diff = evaluate_phase_difference(phase, x1) - evaluate_phase_difference(phase, 0)
        ax.plot(a.wave, diff, c=cmap(norm(x1)))

    fig.colorbar(sm, label='SALT2 $x_1$')

    ax.set_xlabel(a.settings['spectrum_plot_xlabel'])
    ax.set_ylabel('Difference relative to $x_1=0$ (mag)')
    ax.set_title('Difference in interpolation at %+d days' % phase)
    ax.set_ylim(0.4, -0.4)
    a.savefig('time_evolution_x1_difference_phase_%d.pdf' % phase, fig)

if a.settings['differential_evolution_use_salt_x1']:
    for phase in [-5, -3, -1, 1, 3, 5]:
        plot_x1_difference(phase)

## Gray dispersion

In [8]:
# Check if the gray dispersion is phase dependent. This should not be the
# case if the model is working properly.
gray_scale = a.differential_evolution_result['gray_dispersion_scale']
print(f"Gray dispersion scale: {gray_scale:.4f} mag")

plt.figure()
gray_offsets = a.differential_evolution_result['gray_offsets']
plt.scatter(a.salt_phases, gray_offsets, s=3, label='Individual spectra')
math.plot_binned_mean(a.salt_phases, gray_offsets, c='C2', lw=2, label='Binned mean')
math.plot_binned_rms(a.salt_phases, gray_offsets, c='C3', lw=2, label='Binned RMS')
plt.xlabel('SALT2 Phase (days)')
plt.ylabel('Gray offset (mag)')
plt.legend()
a.savefig('gray_offset_vs_phase.pdf')

Gray dispersion scale: 0.0292 mag


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Differential evolution uncertainty

In [9]:
coefs = a.differential_evolution_result['phase_dispersion_coefficients']
num_phase_coefficients = len(coefs)

phase_range = a.settings['phase_range']

def evaluate_phase_dispersion(phase):
    phase_scale = np.abs((num_phase_coefficients / 2) * (phase / phase_range))
    full_bins = int(np.floor(phase_scale))
    remainder = phase_scale - full_bins

    phase_coefficients = np.zeros(num_phase_coefficients)

    for j in range(full_bins + 1):
        if j == full_bins:
            weight = remainder
        else:
            weight = 1
            
        if weight == 0:
            break
            
        if phase > 0:
            phase_bin = num_phase_coefficients // 2 + j
        else:
            phase_bin = num_phase_coefficients // 2 - 1 - j
            
        phase_coefficients[phase_bin] = weight
        
    fractional_dispersion = phase_coefficients.dot(coefs)
    
    # Convert to magnitudes
    mag_dispersion = frac_to_mag(fractional_dispersion)

    return mag_dispersion

phases = np.linspace(-phase_range, phase_range, 1 + num_phase_coefficients)

eval_coefs = np.array([evaluate_phase_dispersion(phase) for phase in phases])

# Uncertainties for different wavelengths
plt.figure()
num_wave = 10
for i in range(num_wave):
    min_wave = a.wave[0]
    max_wave = a.wave[-1]
    wave_range = max_wave - min_wave
    target_wave = min_wave + wave_range * i / (num_wave - 1)
    idx = np.argmin(np.abs(a.wave - target_wave))
    use_wave = a.wave[idx]
    color = plt.cm.rainbow((use_wave - min_wave) / wave_range)
    plt.plot(phases, eval_coefs[:, idx], label='%d $\AA$' % use_wave, c=color)
    
plt.xlim(-5.2, 5.2)
plt.xlabel('Phase (days)')
plt.ylabel('Model uncertainty (mag)')
plt.legend()
a.savefig('time_evolution_uncertainty_phase.pdf')

plt.figure(figsize=a.settings['spectrum_plot_figsize'])
for i in range(len(phases)):
    plt.plot(a.wave, eval_coefs[i], label='%.2f days' % phases[i])
plt.legend()
plt.xlabel(a.settings['spectrum_plot_xlabel'])
plt.ylabel('Model uncertainty (mag)')
a.savefig('time_evolution_uncertainty_wavelength.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Model accuracy

In [10]:
max_flux = a.differential_evolution_result['maximum_flux']
max_fluxerr = a.differential_evolution_result['maximum_fluxerr']

max_magerr = frac_to_mag(max_fluxerr / max_flux)

rbtl_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])

def plot_uncertainties(show_rbtl=False):
    plt.figure(figsize=a.settings['spectrum_plot_figsize'])
    offset = 29
    
    # Make sure that we include the worst offender.
    max_loc = np.argmax(np.sum(max_magerr**2, axis=1))
    start = max_loc % offset
    
    for idx in range(start, len(a.targets), offset):
        plt.plot(a.wave, max_magerr[idx], label=a.targets[idx].name)
    plt.legend(ncol=2)
    
    plt.xlabel('Wavelength ($\AA$)')
    
    if show_rbtl:
        plt.plot(a.wave, rbtl_dispersion, label='SN intrinsic dispersion', c='k', lw=2, ls='--')
        plt.ylabel('Dispersion (mag)')
        filename = 'maximum_uncertainty_rbtl.pdf'
    else:
        plt.ylabel('Uncertainty on $f_{max}$ (mag)')
        filename = 'maximum_uncertainty_norbtl.pdf'
        
    plt.legend(ncol=2)
    a.savefig(filename)

plot_uncertainties(False)
plot_uncertainties(True)


plt.figure(figsize=a.settings['spectrum_plot_figsize'])
for idx in range(len(a.targets)):
    if idx == 0:
        label = 'Individual uncertainties of $f_{max}$'
    else:
        label = ''
    plt.plot(a.wave, max_magerr[idx], label=label, alpha=0.02, c='C0')
plt.plot(a.wave, rbtl_dispersion, label='Supernova intrinsic dispersion', lw=2, ls='--', c='k')
plt.plot(a.wave, np.median(max_magerr, axis=0), label='Median uncertainty on $f_{max}$', lw=2, ls='--', c='C0')
plt.plot(a.wave, np.max(max_magerr, axis=0), label='Maximum uncertainty on $f_{max}$', c='C1')
plt.legend()
plt.ylabel('Dispersion (mag)')
plt.xlabel('Wavelength ($\AA$)')
a.savefig('maximum_uncertainty_median.pdf')

plt.figure(figsize=a.settings['spectrum_plot_figsize'])
plt.plot(a.wave, rbtl_dispersion, label='SN intrinsic dispersion', lw=2, ls='--', c='k')
plt.plot(a.wave, np.min(max_magerr, axis=0), label='Minimum $\sigma_{f,max}$')
for percentile in (25, 50, 75):
    plt.plot(a.wave, np.percentile(max_magerr, percentile, axis=0), label='%dth percentile $\sigma_{f,max}$' % percentile)
plt.plot(a.wave, np.max(max_magerr, axis=0), label='Maximum $\sigma_{f,max}$')
plt.legend(ncol=2)
plt.ylabel('Dispersion (mag)')
plt.xlabel('Wavelength ($\AA$)')
a.savefig('maximum_uncertainty_percentile.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Contribution to the total interpolation uncertainty from various sources

In [11]:
targets = []
diffs = []
phases_1 = []
phases_2 = []
x1s = []
gray_differences = []

gray_offsets = a.differential_evolution_result['gray_offsets']

center_specs = a.spectra[a.center_mask]
center_gray_offsets = gray_offsets[a.center_mask]
for target_idx in range(len(a.targets)):
    near_max_spec = center_specs[target_idx]
    
    target_mask = (a.target_map == target_idx) & (~a.center_mask)
    target_specs = a.spectra[target_mask]
    target_gray_offsets = gray_offsets[target_mask]
    
    for spec_idx, target_spec in enumerate(target_specs):
        phase_diff = target_spec.phase - near_max_spec.phase
        # if np.abs(phase_diff) < 1:
            # continue

        targets.append(a.targets[target_idx])
        diff = -2.5*np.log10(target_spec.flux / near_max_spec.flux)
        diffs.append(diff)
        phases_1.append(near_max_spec.phase)
        phases_2.append(target_spec.phase)
        x1s.append(a.salt_x1[target_idx])
        
        gray_differences.append(target_gray_offsets[spec_idx] - center_gray_offsets[target_idx])

targets = np.array(targets)
diffs = np.array(diffs)
phases_1 = np.array(phases_1)
phases_2 = np.array(phases_2)
x1s = np.array(x1s)
gray_differences = np.array(gray_differences)

phase_diffs = phases_2 - phases_1

def plot_diffs(diffs, model_subtracted=False):
    sel_mask = np.zeros(len(diffs), dtype=bool)
    sel_mask[4::50] = True
    sel_mask[np.abs(phase_diffs) < 1] = False
    
    plt.figure(figsize=a.settings['spectrum_plot_figsize'])
    
    for use_idx in np.where(sel_mask)[0]:
        target = targets[use_idx]
        phase_1 = phases_1[use_idx]
        phase_2 = phases_2[use_idx]
        
        if phase_1 > phase_2:
            phase_1, phase_2 = phase_2, phase_1
        
        label = '%s, %.1f to %.1f days' % (target, phase_1, phase_2)
        
        plt.plot(a.wave, diffs[use_idx] / phase_diffs[use_idx], alpha=0.5, label=label)
        
    plt.legend(ncol=2)

    plt.ylim(-0.25, 0.25)
    plt.xlabel('Wavelength ($\AA$)')
    if model_subtracted:
        plt.ylabel('$\Delta m / \Delta t$ (data)\n - $\Delta m / \Delta t$ (model) (mag/day)')
    else:
        plt.ylabel('$\Delta m / \Delta t$ (data) (mag/day)')

plot_diffs(diffs)
a.savefig('raw_phase_difference.pdf')

residuals_no_x1 = []
residuals_x1 = []
for diff, phase_1, phase_2, x1 in zip(diffs, phases_1, phases_2, x1s):
    model_no_x1 = evaluate_phase_difference(phase_2, 0) - evaluate_phase_difference(phase_1, 0)
    model_x1 = evaluate_phase_difference(phase_2, x1) - evaluate_phase_difference(phase_1, x1)
    residuals_no_x1.append(diff - model_no_x1)
    residuals_x1.append(diff - model_x1)
    
residuals_no_x1 = np.array(residuals_no_x1)
residuals_x1 = np.array(residuals_x1)

residuals_gray_no_x1 = residuals_no_x1 - gray_differences[:, None]
residuals_gray_x1 = residuals_x1 - gray_differences[:, None]

plot_diffs(residuals_gray_no_x1, True)
a.savefig('corr_phase_difference_no_x1.pdf')

plot_diffs(residuals_gray_x1, True)
a.savefig('corr_phase_difference_x1.pdf')

def print_interpolation_residuals(min_days, max_days):
    cut = (np.abs(phase_diffs) < max_days) & (np.abs(phase_diffs) > min_days)

    def do_print(label, vals, cut):
        cut_vals = vals[cut]
        print('%20s: std=%.3f, NMAD=%.3f' % (label, math.rms(cut_vals), math.nmad(cut_vals)))

    print("Interpolation of %.1f-%.1f days:" % (min_days, max_days))
    do_print('Raw', diffs, cut)    
    do_print('Phase', residuals_no_x1, cut)    
    do_print('Phase + x1', residuals_x1, cut)    
    do_print('Phase + gray', residuals_gray_no_x1, cut)    
    do_print('Phase + x1 + gray', residuals_gray_x1, cut)    
    print("")
    
print_interpolation_residuals(0., 1.5)
print_interpolation_residuals(1.5, 2.5)
print_interpolation_residuals(2.5, 5.5)
print_interpolation_residuals(5.5, 10.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Interpolation of 0.0-1.5 days:
                 Raw: std=0.052, NMAD=0.039
               Phase: std=0.045, NMAD=0.034
          Phase + x1: std=0.045, NMAD=0.034
        Phase + gray: std=0.029, NMAD=0.014
   Phase + x1 + gray: std=0.029, NMAD=0.014

Interpolation of 1.5-2.5 days:
                 Raw: std=0.115, NMAD=0.087
               Phase: std=0.080, NMAD=0.066
          Phase + x1: std=0.080, NMAD=0.066
        Phase + gray: std=0.060, NMAD=0.042
   Phase + x1 + gray: std=0.060, NMAD=0.042

Interpolation of 2.5-5.5 days:
                 Raw: std=0.178, NMAD=0.126
               Phase: std=0.094, NMAD=0.075
          Phase + x1: std=0.094, NMAD=0.075
        Phase + gray: std=0.081, NMAD=0.055
   Phase + x1 + gray: std=0.081, NMAD=0.055

Interpolation of 5.5-10.5 days:
                 Raw: std=0.272, NMAD=0.209
               Phase: std=0.127, NMAD=0.098
          Phase + x1: std=0.127, NMAD=0.098
        Phase + gray: std=0.108, NMAD=0.076
   Phase + x1 + gray: std=0.108, NMA

## What fraction of the interpolation uncertainty is common?

In [12]:
phases = np.linspace(-5, 5, 101)

common_dispersion = []
residual_dispersion = []

for phase in phases:
    common_dispersion.append(np.sqrt(np.mean(evaluate_phase_difference(phase)**2)))
    residual_dispersion.append(np.sqrt(np.mean(evaluate_phase_dispersion(phase)**2)))
    
common_dispersion = np.array(common_dispersion)
residual_dispersion = np.array(residual_dispersion)
    
plt.figure()
plt.plot(phases, common_dispersion, label='Common variation (explained)')
plt.plot(phases, residual_dispersion, label='Residual variation (unexplained)')
plt.legend()
plt.xlabel('Phase (days)')
plt.ylabel('Overall dispersion (mag)')

plt.figure()
plt.plot(phases, residual_dispersion**2 / (residual_dispersion**2 + common_dispersion**2), label='Fraction of dispersion remaining')
plt.legend()
plt.xlabel('Phase (days)')
plt.ylabel('Overall dispersion (mag)')

print("Median fraction of variance explained: %.3f" % (1 - np.nanmedian(residual_dispersion**2 / (residual_dispersion**2 + common_dispersion**2))))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Median fraction of variance explained: 0.846


<ipython-input-12-bd488f3d2726>:21: RuntimeWarning: invalid value encountered in true_divide
  plt.plot(phases, residual_dispersion**2 / (residual_dispersion**2 + common_dispersion**2), label='Fraction of dispersion remaining')
<ipython-input-12-bd488f3d2726>:26: RuntimeWarning: invalid value encountered in true_divide
  print("Median fraction of variance explained: %.3f" % (1 - np.nanmedian(residual_dispersion**2 / (residual_dispersion**2 + common_dispersion**2))))


# Reading between the lines plots

## Show spectra before and after

In [16]:
fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
a.plot_flux(ax, a.maximum_flux[a.uncertainty_mask], lw=1, label='Individual spectra')
ax.legend()
a.savefig('spectra_at_maximum.pdf', fig)

fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
a.plot_flux(ax, a.scale_flux[a.uncertainty_mask], lw=1., label='Individual spectra')
a.plot_flux(ax, a.mean_flux, c='k', label='Mean spectrum')
ax.legend()
a.savefig('scale_spectra.pdf', fig)

fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
a.plot_flux(ax, a.mean_flux, a.mean_flux * a.rbtl_result['fractional_dispersion'], label='Mean spectrum', uncertainty_label='Supernova intrinsic dispersion')
ax.legend()
a.savefig('scale_spectra_model.pdf', fig)

fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
intrinsic_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])
ax.plot(a.wave, intrinsic_dispersion, c='k', lw=2, label='Supernova intrinsic dispersion')
ax.legend()
ax.set_xlabel(a.settings['spectrum_plot_xlabel'])
ax.set_ylabel('Intrinsic dispersion (mag)')
ax.set_ylim(0, None)
a.savefig('rbtl_intrinsic_dispersion.pdf', fig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# Combined version

figsize = a.settings['spectrum_plot_figsize_triple']

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=figsize, sharex=True)

a.plot_flux(ax1, a.maximum_flux[::8], lw=0.5, label='Individual spectra', c='C0')
ax1.set_title('Original spectra')
ax1.set_xlabel(None)

a.plot_flux(ax2, a.scale_flux[a.uncertainty_mask][::8], lw=0.5, label='Individual spectra', c='C0')
a.plot_flux(ax2, a.mean_flux, c='k', lw=2, ls='--', label='Mean spectrum')
ax2.set_title('Dereddened spectra')
ax2.set_xlabel(None)

intrinsic_dispersion = frac_to_mag(a.rbtl_result['fractional_dispersion'])
ax3.plot(a.wave, intrinsic_dispersion, c='k')
ax3.set_xlabel(a.settings['spectrum_plot_xlabel'])
ax3.set_ylabel('Intrinsic dispersion (mag)')
ax3.set_title('Recovered intrinsic dispersion')
ax3.set_ylim(0, None)

a.savefig('rbtl_spectra_combined.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Manifold learning plots

## Plot the parameter space

In [4]:
fig = a.scatter_combined([a.embedding[:, 2], a.embedding[:, 1], a.embedding[:, 0]], a.uncertainty_mask, vmin=-4, vmax=4, label='Other Component Value')
a.savefig('embedding_components.pdf', fig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Reconstruction uncertainty

In [14]:
# Note: the total variance isn't defined for Isomap. The variances of the transfomed components
# do map onto the variance of real components though. We provide a very rough estimate of the
# measurement variance for comparison purposes... not sure how much it can be trusted...

num_show = 10

# Do an initial embedding with as many components as possible to get the full variance.
embedding = a.generate_embedding(num_components=50)
variances = np.var(embedding[a.uncertainty_mask], axis=0)

ref_var = np.sum(variances[:10])

plot_ref = variances[0]

plt.figure()
plt.scatter(np.arange(num_show), variances[:num_show] / plot_ref, label='Contributed variance of each component')
plt.ylim(0, None)
plt.xlabel('Component number')
plt.ylabel('Relative variance explained')
plt.xticks(np.arange(num_show), np.arange(num_show) + 1)

a.savefig('isomap_component_variance.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Twin reconstruction

In [15]:
from IPython.display import display
from scipy.spatial.distance import pdist
from scipy.stats import percentileofscore
import pandas as pd

def plot_twin_distances(embedding, twins_percentile=10, figsize=None):
    """Plot a histogram of where twins show up in the transformed
    embedding.
    """

    mask = a.uncertainty_mask

    spec_dists = pdist(a.fractional_differences[mask])
    embedding_dists = pdist(embedding[mask])

    splits = {
        "Best 10% of twinness": (0, 10),
        "10-20%": (10, 20),
        "20-50%": (20, 50),
        "Worst 50% of twinness": (50, 100),
    }

    # Set weight so that the histogram is 1 if we have every element in
    # that bin.
    weight = 100 / len(embedding_dists)

    all_percentiles = []
    all_weights = []

    all_spec_cuts = []
    all_embedding_cuts = []

    for label, (min_percentile, max_percentile) in splits.items():
        spec_cut = (spec_dists >= np.percentile(spec_dists, min_percentile)) & (
            spec_dists < np.percentile(spec_dists, max_percentile)
        )
        embedding_cut = (embedding_dists >= np.percentile(embedding_dists, min_percentile)) & (
            embedding_dists < np.percentile(embedding_dists, max_percentile)
        )
        percentiles = []
        for dist in embedding_dists[spec_cut]:
            percentiles.append(percentileofscore(embedding_dists, dist))
        percentiles = np.array(percentiles)
        weights = np.ones(len(percentiles)) * weight

        all_percentiles.append(percentiles)
        all_weights.append(weights)
        all_spec_cuts.append(spec_cut)
        all_embedding_cuts.append(embedding_cut)

    plt.figure(figsize=figsize)
    plt.hist(
        all_percentiles,
        100,
        (0, 100),
        weights=all_weights,
        histtype="barstacked",
        label=splits.keys(),
    )
    plt.xlabel("Recovered twinness percentile in the embedded space")
    plt.ylabel("Fraction in bin")
    plt.legend()

    plt.xlim(0, 100)
    plt.ylim(0, 1)

    for label, (min_percentile, max_percentile) in splits.items():
        plt.axvline(max_percentile, c="k", lw=2, ls="--")

    # Build leakage matrix.
    leakage_matrix = np.zeros((len(splits), len(splits)))
    for idx_1, label_1 in enumerate(splits.keys()):
        for idx_2, label_2 in enumerate(splits.keys()):
            spec_cut = all_spec_cuts[idx_1]
            embedding_cut = all_embedding_cuts[idx_2]
            leakage = np.sum(embedding_cut & spec_cut) / np.sum(spec_cut)
            leakage_matrix[idx_1, idx_2] = leakage

    # Print the leakage matrix using pandas
    df = pd.DataFrame(
        leakage_matrix,
        index=["From %s" % i for i in splits.keys()],
        columns=["To %s" % i for i in splits.keys()],
    )
    display(df)

    return leakage_matrix

In [16]:
# Plot where twins and non-twins end up for different number of components.
# We also make a summary plot.
confused_fraction = []

plot_components = np.arange(1, 6)
for num_components in plot_components:
    embedding = a.generate_embedding(num_components=num_components)
    leakage_matrix = plot_twin_distances(embedding)
    if num_components == 1:
        title = '1 Component + Color'
    else:
        title = '%d Components + Color' % num_components
    plt.title(title)
    plt.xlabel('Recovered twinness percentile in the embedded space')
    a.savefig('twins_recovery_%d_components.pdf' % num_components)

    confused_fraction.append(leakage_matrix[3, 0] + leakage_matrix[3, 1])

plt.figure()
plt.scatter(np.arange(len(confused_fraction)) + 1, confused_fraction)
plt.xticks(plot_components, plot_components)
plt.ylim(0, 0.1)
plt.xlabel('Number of components (in addition to color)')
plt.ylabel('Fraction of non-twins confused as twins')
a.savefig('twins_confusion.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,To Best 10% of twinness,To 10-20%,To 20-50%,To Worst 50% of twinness
From Best 10% of twinness,0.282930,0.272849,0.438172,0.006048
From 10-20%,0.170027,0.184140,0.548387,0.097446
From 20-50%,0.114497,0.110688,0.409814,0.365001
From Worst 50% of twinness,0.040737,0.042216,0.156763,0.760285


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,To Best 10% of twinness,To 10-20%,To 20-50%,To Worst 50% of twinness
From Best 10% of twinness,0.581989,0.311156,0.106855,0.000000
From 10-20%,0.188844,0.306452,0.504032,0.000672
From 20-50%,0.057809,0.105310,0.599821,0.237060
From Worst 50% of twinness,0.011159,0.013310,0.117908,0.857623


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,To Best 10% of twinness,To 10-20%,To 20-50%,To Worst 50% of twinness
From Best 10% of twinness,0.745296,0.224462,0.030242,0.000000
From 10-20%,0.191532,0.461694,0.346774,0.000000
From 20-50%,0.019942,0.102173,0.716558,0.161326
From Worst 50% of twinness,0.000672,0.001479,0.094649,0.903200


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

,To Best 10% of twinness,To 10-20%,To 20-50%,To Worst 50% of twinness
From Best 10% of twinness,0.760753,0.206989,0.032258,0.000000
From 10-20%,0.190188,0.463038,0.346774,0.000000
From 20-50%,0.015685,0.107327,0.721040,0.155949
From Worst 50% of twinness,0.000403,0.001613,0.091557,0.906292


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot steps through component values

In [17]:
def plot_steps(ax, component, num_steps=20, colorbar_aspect=50):
    mask = a.uncertainty_mask

    use_embedding = a.embedding[mask, component]
    use_flux = a.scale_flux[mask]

    bin_edges = np.percentile(use_embedding, np.linspace(0, 100, num_steps + 1))
    bin_centers = (bin_edges[1:] + bin_edges[:-1]) / 2.

    sm = plt.cm.ScalarMappable(cmap=a.settings['colormap'], norm=plt.Normalize(vmin=bin_centers[0], vmax=bin_centers[-1]))
    sm._A = []

    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)

        # Make the extreme values of components get plotted on top if everything overlaps.
        zorder = np.abs(mean_val)

        a.plot_flux(ax, step_flux, c=sm.to_rgba(mean_val), zorder=zorder)

    fig.colorbar(sm, ax=ax, orientation='vertical', label='Component Value', pad=0, aspect=colorbar_aspect)

In [18]:
# All plots combined
# fig, axes = plt.subplots(3, 1, figsize=(a.settings['spectrum_plot_figsize'][0], a.settings['spectrum_plot_figsize'][1] * 3 - 1.5), sharex=True)
fig, axes = plt.subplots(3, 1, figsize=a.settings['spectrum_plot_figsize_triple'], sharex=True)

for component, ax in enumerate(axes):
    plot_steps(ax, component, colorbar_aspect=17)

    if component != 2:
        ax.set_xlabel(None)

    ax.set_title('Component %d' % (component + 1))
    # ax.set_ylabel('Normalized flux\n(erg/cm$^2$/s/Hz)')

a.savefig('component_steps_combined.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
# Version for talks
fig, axes = plt.subplots(3, 1, figsize=(9, 7.5), sharex=True)

for component, ax in enumerate(axes):
    plot_steps(ax, component, colorbar_aspect=17)

    if component != 2:
        ax.set_xlabel(None)

    ax.set_ylabel('Flux')

a.savefig('component_steps_combined_talks.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
# Single plots
for component in range(a.settings['isomap_num_components']):
    fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
    ax.set_title(f'Component {component+1}')
    plot_steps(ax, component)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Comparision to Branch classifications

In [23]:
branch_colors = {
    'Broad Line': 'b',
    'Cool': 'r',
    'Core Normal': 'black',
    'Shallow Silicon': 'green',
}

from matplotlib.colors import ListedColormap
cmap = ListedColormap(branch_colors.values())
color_id_map = {j:i for i, j in enumerate(branch_colors)}
colors = np.array([color_id_map[i] for i in a.spectral_indicators['branch_label']])

fig, ax = plt.subplots()
m = a.uncertainty_mask
scatter = ax.scatter(
    a.spectral_indicators['EWSiII6355'][m],
    a.spectral_indicators['EWSiII5972'][m],
    c=colors[m],
    cmap=cmap,
    s=a.settings['scatter_plot_marker_size'],
    edgecolors='gray'
)
ax.set_xlabel('SiII 6355 $\AA$ Equivalent Width')
ax.set_ylabel('SiII 5972 $\AA$ Equivalent Width')

ax.legend(handles=scatter.legend_elements()[0], labels=branch_colors.keys())

a.savefig('branch_classification.pdf', fig)

a.scatter_combined(a.spectral_indicators['branch_label'], a.uncertainty_mask, discrete_color_map=branch_colors, wspace=0.05, hspace=0.05)
a.savefig('branch_labels.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Variation of Core Normal SNe Ia

In [24]:
core_normal_cut = a.spectral_indicators['branch_label'] == 'Core Normal'

component = 0

cut_flux = a.scale_flux[core_normal_cut]
cut_coord = a.embedding[core_normal_cut][:, component]

sort_flux = cut_flux[np.argsort(cut_coord)]
sort_coord = cut_coord[np.argsort(cut_coord)]

num_bins = 10

cmap = a.settings['colormap']
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=np.percentile(cut_coord, 100 / num_bins / 2), vmax=np.percentile(cut_coord, 100 * (1 - 1. / num_bins / 2))))
sm._A = []

def plot_spec(bin_idx, ax):
    min_idx = int(len(sort_coord) / num_bins * bin_idx)
    max_idx = int(len(sort_coord) / num_bins * (bin_idx + 1))
    bin_flux = sort_flux[min_idx:max_idx]
    
    f = np.median(bin_flux, axis=0)
    
    mean_val = np.mean(sort_coord[min_idx:max_idx])
    
    a.plot_flux(ax, f, c=sm.to_rgba(mean_val), zorder=np.abs(mean_val))
    # plt.plot(a.wave, f * spectrum_plot_scale, c=sm.to_rgba(mean_val), zorder=np.abs(mean_val))
    
fig, ax = plt.subplots(figsize=a.settings['spectrum_plot_figsize'])
for i in range(num_bins):
    plot_spec(i, ax)

fig.colorbar(sm, ax=ax, orientation='vertical', label='Value of Component %d' % (component + 1), pad=0, aspect=50)

a.savefig('core_normal_comparison.pdf', fig)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Peculiar SNe Ia

In [16]:
peculiar_colors = {
    'Normal': 'lightgray',
    '91T-like': 'C0',
    '91bg-like': 'C1',
    '02cx-like': 'C2',
}

a.scatter_combined(a.peculiar_data['kind'], a.uncertainty_mask, discrete_color_map=peculiar_colors)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Recovering other indicators of intrinsic diversity

In [48]:
# Nordin colors
nordin_bands = {
    'uNi': (3300., 3510.),
    'uTi': (3510., 3660.),
    'uSi': (3660., 3760.),
    'uCa': (3750., 3860.),
}

band = 'uCa'

wave_min, wave_max = nordin_bands[band]
cut = (a.wave > wave_min) & (a.wave < wave_max)
values = -2.5*np.log10(np.sum(a.scale_flux[:, cut], axis=1) / np.sum(a.mean_flux[cut]))

a.scatter_combined(values, a.uncertainty_mask, vmin=-0.3, vmax=0.3, label='uCa (mag)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
# Set up an array to hold everything
all_indicators = []

In [18]:
# Spectral features
indicators = a.spectral_indicators

name_map = {
    'EWCaIIHK': 'pEW Ca II HK',
    'EWSiII4000': 'pEW Si II 4000',#$\AA$',
    'EWSiII5972': 'pEW Si II 5972',#$\AA$',
    'EWSiII6355': 'pEW Si II 6355',#$\AA$',
    'vCaIIHK': 'Velocity Ca II HK',
    'vSiII6355': 'Velocity Si II 6355',#$\AA$',
    'lamCaIIHK': 'Lambda Ca II HK',
    'lamSiII6355': 'Lambda Si II 6355',#$\AA$',
}

for key in indicators.keys():
    if key not in name_map:
        continue

    values = indicators[key]

    if key[:3] == 'lam':
        continue
        
    if key[:1] == 'v':
        # Use km/s and make everything positive.
        values = values / -1000

    all_indicators.append((name_map[key], values, a.uncertainty_mask))

In [19]:
# SALT2 X1
all_indicators.append(('SALT2 $x_1$', a.salt_x1, a.salt_mask))

In [20]:
import pickle

# Sugar parameters
pickle_data =  open('./data/sugar_parameters.pkl').read().replace('\r\n', '\n').encode('latin1')
sugar_data = pickle.loads(pickle_data, encoding='latin1')

sugar_rows = []
for target in a.targets:
    try:
        row = sugar_data[target.name.encode('latin1')]
        sugar_rows.append([row['q1'], row['q2'], row['q3']])
    except KeyError:
        sugar_rows.append([np.nan] * 3)

sugar_embedding = np.array(sugar_rows, dtype=float)
sugar_mask = ~np.isnan(sugar_embedding[:, 0])

for component in range(3):
    all_indicators.append(('SUGAR Component %d\n(Leget et al. 2019)' % (component + 1), sugar_embedding[:, component], sugar_mask))

In [21]:
# Nordin colors
nordin_bands = {
    'uNi': (3300., 3510.),
    'uTi': (3510., 3660.),
    'uSi': (3660., 3760.),
    'uCa': (3750., 3860.),
}

for label, (wave_min, wave_max) in nordin_bands.items():
    cut = (a.wave > wave_min) & (a.wave < wave_max)
    
    values = -2.5*np.log10(np.sum(a.scale_flux[:, cut], axis=1) / np.sum(a.mean_flux[cut]))
    
    all_indicators.append(('U-band variation—%s\n(Nordin et al. 2018)' % label, values, a.uncertainty_mask))

In [22]:
# SNEMO parameters
import pandas as pd
snemo_data = pd.read_csv('./data/snemo_salt_coefficients_snf.csv').set_index('SN')

nan_row = snemo_data.iloc[0].copy()
nan_row[:] = np.nan

snemo_rows = []
for target in a.targets:
    try:
        row = snemo_data.loc[target.name]
        snemo_rows.append(row)
    except KeyError:
        snemo_rows.append(nan_row)

snemo_embedding = pd.DataFrame(snemo_rows)
snemo_mask = ~np.isnan(snemo_embedding['salt_c'])
snemo_labels = snemo_data.columns

# SNEMO 2
all_indicators.append(('SNEMO2 Component 1\n(Saunders et al. 2018)', snemo_embedding['snemo2_c1'], snemo_mask))

# SNEMO 7
for component in range(6):
    all_indicators.append(('SNEMO7 Component %d' % (component + 1), snemo_embedding[f'snemo7_c{component+1}'], snemo_mask))
    
# SNEMO 15
# for component in range(14):
    # all_indicators.append(('SNEMO15 Component %d' % (component + 1), snemo_embedding[f'snemo15_c{component+1}'], snemo_mask))

In [23]:
def find_rotation(values, mask):
    # Find the best predictor of an indicator
    def to_min(x):
        diff = values - a.embedding.dot(x[1:]) - x[0]
        return np.sum(diff[mask]**2)

    res = minimize(to_min, [0, 0, 0, 0])

    rotation = res.x[1:] / np.sqrt(np.sum(res.x[1:]**2))
    best_guess = a.embedding.dot(res.x[1:]) + res.x[0]
    
    return rotation, best_guess

def find_rotation_quadratic(values, mask):
    # Find the best predictor of an indicator
    def evaluate(x):
        e = a.embedding.T
        
        model = (
            x[0]
            + e[0] * x[1]
            + e[1] * x[2]
            + e[2] * x[3]
            + e[0] * e[0] * x[4]
            + e[1] * e[1] * x[5]
            + e[2] * e[2] * x[6]
            + e[0] * e[1] * x[7]
            + e[0] * e[2] * x[8]
            + e[1] * e[2] * x[9]
        )
        
        return model
        
    def to_min(x):
        model = evaluate(x)
        diff = values - model
        return np.sum(diff[mask]**2)

    res = minimize(to_min, [0] * 10)

    best_guess = evaluate(res.x)
    
    return res.x, best_guess

names = []
data = []

for name, values, mask in all_indicators[::-1]:
    mask = mask & a.uncertainty_mask
    
    rotation, best_guess = find_rotation(values, mask)
    corrcoef = np.corrcoef(best_guess[mask], values[mask])[0, 1]
    
    params_quadratic, best_guess_quadratic = find_rotation_quadratic(values, mask)
    corrcoef_quadratic = np.corrcoef(best_guess_quadratic[mask], values[mask])[0, 1]
    
    names.append(name)
    data.append(np.hstack([rotation, corrcoef, corrcoef_quadratic]))

    # print(f'{name} & {rotation[0]:.2f} & {rotation[1]:.2f} & {rotation[2]:.2f} & {corrcoef:.2f}')
    
data = np.array(data)

fig, axes = plt.subplots(1, 2, sharey=True, figsize=(5.0, 8))
plt.subplots_adjust(wspace=0., hspace=0.)

cmap = plot_cmap
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=-1, vmax=1))
sm._A = []

def do_plot(ax, ax_data, xlabels, ylabels=None):
    im = ax.imshow(ax_data, interpolation='nearest', cmap=cmap, vmin=-1, vmax=1)
    ax.set(
        xticks=np.arange(ax_data.shape[1]),
        yticks=np.arange(ax_data.shape[0]),
        xticklabels=xlabels,
    )

    if ylabels is not None:
        ax.set_yticklabels(ylabels)
    else:
        ax.tick_params(axis='y', which='both', left=False, right=False)

    ax.set_xlim(-0.5, ax_data.shape[1] - 0.5)
    ax.set_ylim(-0.5, ax_data.shape[0] - 0.5)

    # Ticks on top
    ax.get_xaxis().set_ticks_position('top')
    
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=70, ha="left", va='center',
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = 0.65
    for i in range(ax_data.shape[0]):
        for j in range(ax_data.shape[1]):
            ax.text(j, i, format(ax_data[i, j], fmt),
                    ha="center", va="center",
                    color="white" if np.abs(ax_data[i, j]) > thresh else "black")
            
do_plot(axes[0], data[:, :3], ['Component 1 Rotation', 'Component 2 Rotation', 'Component 3 Rotation'], names)
do_plot(axes[1], data[:, 3:], ['Linear Transformation\nPearson Correlation', 'Quadratic Transformation\nPearson Correlation'])

fig.colorbar(sm, ax=ax, orientation='vertical')

plt.savefig('./figures/indicators_recovery.pdf')

NameError: name 'minimize' is not defined

In [111]:
from ipywidgets import interact

def plot_component(idx, quadratic=False):
    name, values, mask = all_indicators[idx]

    mask = mask & a.uncertainty_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
        
    min_val = np.min([values[mask], best_guess[mask]])
    max_val = np.max([values[mask], best_guess[mask]])
    val_range = max_val - min_val
    
    plt.figure(figsize=(4, 3.8))
    plt.scatter(best_guess[mask], values[mask])
    
    plt.text(min_val + 0.*val_range, max_val - 0.02*val_range, "$\\rho = %.2f$" % np.corrcoef([values[mask], best_guess[mask]])[0, 1])
    print("$\\rho = %.2f$" % np.corrcoef([values[mask], best_guess[mask]])[0, 1])
    
    plt.plot([min_val - val_range, min_val + val_range], [min_val - val_range, min_val + val_range], c='k', ls='--')

    plt.xlabel('Transformation of the Isomap latent space')
    
    plt.ylabel(name)
    # plt.ylabel("Original measurement")
    # plt.title(name)
    
    plt.xlim(min_val-0.05*val_range, max_val+0.05*val_range)
    plt.ylim(min_val-0.05*val_range, max_val+0.05*val_range)
    
    
interact(plot_component, idx=(0, len(data) - 1), quadratic=True)

ValueError: value must be between min and max (min=0, value=-1, max=-1)

In [ ]:
def plot_component(idx, quadratic=False):
    name, values, mask = all_indicators[idx]

    mask = mask & a.uncertainty_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
        
    min_val = np.min([values[mask], best_guess[mask]])
    max_val = np.max([values[mask], best_guess[mask]])
    val_range = max_val - min_val
    
    plt.figure(figsize=(2.5, 2.7))
    plt.scatter(best_guess[mask], values[mask], s=10)
    
    plt.text(min_val + 0.*val_range, max_val - 0.04*val_range, "$\\rho = %.2f$" % np.corrcoef([values[mask], best_guess[mask]])[0, 1])
    print("$\\rho = %.2f$" % np.corrcoef([values[mask], best_guess[mask]])[0, 1])
    
    plt.plot([min_val - val_range, min_val + val_range], [min_val - val_range, min_val + val_range], c='k', ls='--')

    # plt.xlabel('Transformation of the Isomap latent space')
    plt.xticks([])
    plt.yticks([])
    
    # plt.ylabel(name)
    # plt.ylabel("Original measurement")
    plt.title(name.split('\n')[0])
    
    plt.xlim(min_val-0.05*val_range, max_val+0.05*val_range)
    plt.ylim(min_val-0.05*val_range, max_val+0.05*val_range)

interact(plot_component, idx=(0, len(data) - 1), quadratic=True)

In [ ]:
for i in range(len(data)):
    plot_component(i, quadratic=True)
    plt.savefig('./figures/rotation_%d.png' % i)

In [ ]:
plt.ylabel('SALT2 $x_1$')

In [ ]:
plt.figure()
plt.text(-0.3, 0.6, 'hi')
plt.show()

In [ ]:
plot_component(6, quadratic=True)

In [ ]:
from ipywidgets import interact

def plot_component(idx, quadratic=False):
    name, values, mask = all_indicators[idx]

    mask = mask & a.uncertainty_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
    
    plt.figure()
    plt.scatter(best_guess[mask], values[mask])
    plt.title(name)
    plt.xlabel('Transformation of the Isomap latent space')
    plt.ylabel(name)
    
interact(plot_component, idx=(0, len(data) - 1))

## Figure out what correlation coefficient we would expect

In [16]:
all_corrcoef_linear = []
all_corrcoef_quadratic = []

for i in tqdm.tqdm(range(200)):
    mask = a.interp_mask
    values = np.random.normal(size=len(values))
    
    rotation, best_guess = find_rotation(values, mask)
    corrcoef = np.corrcoef(best_guess[mask], values[mask])[0, 1]
    all_corrcoef_linear.append(corrcoef)
    
    rotation, best_guess = find_rotation_quadratic(values, mask)
    corrcoef = np.corrcoef(best_guess[mask], values[mask])[0, 1]
    all_corrcoef_quadratic.append(corrcoef)

with open('./latex/correlation_simulation.tex', 'w') as f:
    latex_command(f, 'correlationmeanlinear', '%.2f', np.mean(all_corrcoef_linear))
    latex_command(f, 'correlationstdlinear', '%.2f', np.std(all_corrcoef_linear))
    latex_command(f, 'correlationmeanquadratic', '%.2f', np.mean(all_corrcoef_quadratic))
    latex_command(f, 'correlationstdquadratic', '%.2f', np.std(all_corrcoef_quadratic))

NameError: name 'tqdm' is not defined

## Plot rotated spectra

In [17]:
from ipywidgets import interact

def plot_steps(idx, quadratic=False):
    num_steps = 10
    name, values, mask = all_indicators[idx]

    mask = mask & a.interp_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
    
    use_embedding = best_guess[mask]
    use_flux = a.scale_flux[mask]
    
    min_embedding = np.percentile(use_embedding, 5)
    max_embedding = np.percentile(use_embedding, 95)
    
    bin_edges = np.linspace(min_embedding, max_embedding, num_steps+1)
    
    bin_edges[0] = -1e20
    bin_edges[-1] = 1e20
    
    plt.figure(figsize=spectrum_plot_figsize)
    
    cmap = plot_cmap
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=min_embedding, vmax=max_embedding))
    sm._A = []

    for step in range(num_steps):
        step_mask = (use_embedding >= bin_edges[step]) & (use_embedding < bin_edges[step+1])
        step_embedding = use_embedding[step_mask]

        mean_val = np.mean(step_embedding)
        step_flux = np.median(use_flux[step_mask], axis=0)
        
        plt.plot(a.wave, step_flux * spectrum_plot_scale, c=sm.to_rgba(mean_val))
        
    plt.colorbar(sm, label='Rotated Isomap vSi II 6355$\AA$\nEstimate ($10^3$ km/s)')
    
    plt.xlabel('Wavelength ($\AA$)')
    plt.ylabel(spectrum_plot_ylabel)
    plt.ylim(0, None)
    plt.title(name)
    
    
interact(plot_steps, idx=(0, len(data) - 1))

ValueError: value must be between min and max (min=0, value=-1, max=-1)

## Linear combinations of SUGAR components

In [18]:
all_indicators.append(('Isomap Component 1', a.embedding[:, 0], a.interp_mask))
all_indicators.append(('Isomap Component 2', a.embedding[:, 1], a.interp_mask))
all_indicators.append(('Isomap Component 3', a.embedding[:, 2], a.interp_mask))

AttributeError: 'ManifoldTwinsAnalysis' object has no attribute 'interp_mask'

In [19]:
def find_rotation(values, mask):
    # Find the best predictor of an indicator
    def to_min(x):
        diff = values - sugar_embedding.dot(x[1:]) - x[0]
        return np.sum(diff[mask]**2)

    res = minimize(to_min, [0, 0, 0, 0])

    rotation = res.x[1:] / np.sqrt(np.sum(res.x[1:]**2))
    best_guess = sugar_embedding.dot(res.x[1:]) + res.x[0]
    
    return rotation, best_guess

def find_rotation_quadratic(values, mask):
    # Find the best predictor of an indicator
    def evaluate(x):
        e = sugar_embedding.T
        
        model = (
            x[0]
            + e[0] * x[1]
            + e[1] * x[2]
            + e[2] * x[3]
            + e[0] * e[0] * x[4]
            + e[1] * e[1] * x[5]
            + e[2] * e[2] * x[6]
            + e[0] * e[1] * x[7]
            + e[0] * e[2] * x[8]
            + e[1] * e[2] * x[9]
        )
        
        return model
        
    def to_min(x):
        model = evaluate(x)
        diff = values - model
        return np.sum(diff[mask]**2)

    res = minimize(to_min, [0] * 10)

    best_guess = evaluate(res.x)
    
    return res.x, best_guess

names = []
data = []

for name, values, mask in all_indicators[::-1]:
    mask = mask & sugar_mask
    
    rotation, best_guess = find_rotation(values, mask)
    corrcoef = np.corrcoef(best_guess[mask], values[mask])[0, 1]
    
    params_quadratic, best_guess_quadratic = find_rotation_quadratic(values, mask)
    corrcoef_quadratic = np.corrcoef(best_guess_quadratic[mask], values[mask])[0, 1]
    
    names.append(name)
    data.append(np.hstack([rotation, corrcoef, corrcoef_quadratic]))

    # print(f'{name} & {rotation[0]:.2f} & {rotation[1]:.2f} & {rotation[2]:.2f} & {corrcoef:.2f}')
    
data = np.array(data)

fig, axes = plt.subplots(1, 2, sharey=True, figsize=(5.0, 8))
plt.subplots_adjust(wspace=0., hspace=0.)

cmap = plot_cmap
sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=-1, vmax=1))
sm._A = []

def do_plot(ax, ax_data, xlabels, ylabels=None):
    im = ax.imshow(ax_data, interpolation='nearest', cmap=cmap, vmin=-1, vmax=1)
    ax.set(
        xticks=np.arange(ax_data.shape[1]),
        yticks=np.arange(ax_data.shape[0]),
        xticklabels=xlabels,
    )

    if ylabels is not None:
        ax.set_yticklabels(ylabels)
    else:
        ax.tick_params(axis='y', which='both', left=False, right=False)

    ax.set_xlim(-0.5, ax_data.shape[1] - 0.5)
    ax.set_ylim(-0.5, ax_data.shape[0] - 0.5)

    # Ticks on top
    ax.get_xaxis().set_ticks_position('top')
    
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=70, ha="left", va='center',
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = 0.65
    for i in range(ax_data.shape[0]):
        for j in range(ax_data.shape[1]):
            ax.text(j, i, format(ax_data[i, j], fmt),
                    ha="center", va="center",
                    color="white" if np.abs(ax_data[i, j]) > thresh else "black")
            
do_plot(axes[0], data[:, :3], ['Component 1 Rotation', 'Component 2 Rotation', 'Component 3 Rotation'], names)
do_plot(axes[1], data[:, 3:5], ['Linear Rotation\nPearson Correlation', 'Quadratic Rotation\nPearson Correlation'])

fig.colorbar(sm, ax=ax, orientation='vertical')

# plt.savefig('./figures/indicators_recovery.pdf')

NameError: name 'minimize' is not defined

In [20]:
from ipywidgets import interact

def plot_component(idx, quadratic=False):
    name, values, mask = all_indicators[idx]

    mask = mask & sugar_mask
    if quadratic:
        rotation, best_guess = find_rotation_quadratic(values, mask)
    else:
        rotation, best_guess = find_rotation(values, mask)
    
    plt.figure()
    plt.scatter(best_guess[mask], values[mask], c=a.colors[mask], vmin=-0.2, vmax=0.2)
    plt.title(name)
    
interact(plot_component, idx=(0, len(data) - 1))

ValueError: value must be between min and max (min=0, value=-1, max=-1)

## Color comparison

In [8]:
a.scatter_combined(a.rbtl_colors, a.uncertainty_mask, vmin=-0.3, vmax=0.3, label='RBTL Color')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Latex variable dump

In [ ]:
# Define a bunch of functions to make things easier.
def latex_host_step(file, name, var, mags, mask):
    step, step_err = analyze_host_variable(var, mags, mask, plot=False)
    latex_command(file, name, '%.3f $\\pm$ %.3f', (np.abs(step), step_err))

a.fit_gp(kind='salt_raw', verbose=False)
salt_isomap_mags = a.corr_mags.copy()

a.fit_gp(verbose=False)
rbtl_isomap_mags = a.corr_mags.copy()

with open('latex/commands.tex', 'w') as f:
    latex_print(f, "")
    latex_command(f, 'numdatasetsne', '%d', len(a.dataset.targets))
    latex_command(f, 'numdatasetspectra', '%d', np.sum([len(i.spectra) for i in a.dataset.targets]))
    latex_print(f, "")
    latex_command(f, 'nummanifoldsne', '%d', len(a.targets))
    latex_command(f, 'nummanifoldspectra', '%d', len(a.spectra))
    latex_command(f, 'numinterpsne', '%d', np.sum(a.interp_mask))
    latex_print(f, "")
    latex_command(f, 'numsnftrain', '%d', np.sum([i.subset == 'training' for i in a.targets[a.interp_mask]]))
    latex_command(f, 'numsnfvalid', '%d', np.sum([i.subset == 'validation' for i in a.targets[a.interp_mask]]))
    latex_command(f, 'numsnfother', '%d', np.sum([i.subset not in ['training', 'validation'] for i in a.targets[a.interp_mask]]))
    latex_print(f, "")
    latex_command(f, 'numsnredshift', '%d', np.sum(a.interp_mask & (a.redshift_errs >= 0.004)))
    latex_command(f, 'numlowredshift', '%d', np.sum(a.interp_mask & (a.redshifts <= 0.02)))
    latex_command(f, 'numhighav', '%d', np.sum(a.interp_mask & (a.colors - np.nanmedian(a.colors) >= 0.5)))
    latex_print(f, "")
    latex_command(f, 'nummagsne', '%d', np.sum(a.interp_mask & a.redshift_color_mask))
    latex_command(f, 'nummagsnetrain', '%d', np.sum(a.good_mag_mask))
    latex_command(f, 'nummagsnevalidation', '%d', np.sum(a.interp_mask & a.redshift_color_mask & ~a.good_mag_mask))
    latex_print(f, "")
    latex_command(f, 'saltparammb', '%.2f', a.salt_MB)
    latex_command(f, 'saltparamalpha', '%.3f', a.salt_alpha)
    latex_command(f, 'saltparambeta', '%.2f', a.salt_beta)
    latex_command(f, 'saltparamsigmaint', '%.3f', a.salt_intrinsic_dispersion)
    # latex_command(f, 'saltparamrms', '%.3f', np.std(a.salt_hr[a.good_salt_mask]))
    latex_std(f, 'saltparamrms', a.salt_hr[a.good_salt_mask])
    latex_nmad(f, 'saltparamnmad', a.salt_hr[a.good_salt_mask])
    latex_command(f, 'saltparamwrms', '%.3f', a.salt_wrms)
    latex_command(f, 'saltparammindisp', '%.2f', np.min(a.salt_hr_uncertainties[a.good_salt_mask]))
    latex_command(f, 'saltparammaxdisp', '%.2f', np.max(a.salt_hr_uncertainties[a.good_salt_mask]))
    latex_print(f, "")
    latex_std(f, 'rawrbtlmagstd', a.mags[a.good_mag_mask])
    latex_nmad(f, 'rawrbtlmagnmad', a.mags[a.good_mag_mask])
    # latex_print(f, "")
    # latex_command(f, 'twinrbtlmagstd', '%.3f', a.twins_rms)
    # latex_command(f, 'twinrbtlmagnmad', '%.3f', a.twins_nmad)
    latex_print(f, "")
    latex_std(f, 'saltcomprawrbtlmagstd', a.mags[a.good_mag_mask & a.good_salt_mask])
    latex_std(f, 'saltcompsaltmagstd', a.salt_hr[a.good_mag_mask & a.good_salt_mask])

    a.fit_gp(verbose=False, kind='salt_raw')
    gp_uncertainties = np.sqrt(np.diag(a.gp_hyperparameter_covariance))
    latex_print(f, "")
    latex_command(f, 'saltgpcolor', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[0], gp_uncertainties[0]))
    latex_command(f, 'saltgpintdisp', '%.3f $\\pm$ %.3f', (a.gp_hyperparameters[1], gp_uncertainties[1]))
    latex_command(f, 'saltgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(a.gp_hyperparameters[2]), gp_uncertainties[2]))
    latex_command(f, 'saltgpkernellengthscale', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[3], gp_uncertainties[3]))
    latex_std(f, 'saltgprms', a.corr_mags[a.good_salt_mask & a.interp_mask])
    latex_std(f, 'saltgpcompsaltrms', a.salt_hr[a.good_salt_mask & a.interp_mask])

    a.fit_gp(verbose=False)
    gp_uncertainties = np.sqrt(np.diag(a.gp_hyperparameter_covariance))
    latex_print(f, "")
    latex_command(f, 'rbtlgpcolor', '%.2f $\\pm$ %.2f', (a.fiducial_rv * (1 + a.gp_hyperparameters[0]), a.fiducial_rv * gp_uncertainties[0]))
    latex_command(f, 'rbtlgpintdisp', '%.3f $\\pm$ %.3f', (a.gp_hyperparameters[1], gp_uncertainties[1]))
    latex_command(f, 'rbtlgpkernelamp', '%.3f $\\pm$ %.3f', (np.abs(a.gp_hyperparameters[2]), gp_uncertainties[2]))
    latex_command(f, 'rbtlgpkernellengthscale', '%.2f $\\pm$ %.2f', (a.gp_hyperparameters[3], gp_uncertainties[3]))

    latex_print(f, "")
    latex_std(f, 'rbtlgprms', a.corr_mags[a.good_mag_mask])
    latex_command(f, 'rbtlgpnmad', '%.3f', math.nmad(a.corr_mags[a.good_mag_mask]))

    latex_print(f, "")
    x1 = a.salt_hr[(a.embedding[:, 0] < 3) & a.good_salt_mask & a.interp_mask]
    x2 = a.salt_hr[(a.embedding[:, 0] > 3) & a.good_salt_mask & a.interp_mask]
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    err1 = np.std(x1) / np.sqrt(len(x1))
    err2 = np.std(x2) / np.sqrt(len(x2))
    latex_command(f, 'saltisomapdiff', '%.3f $\\pm$ %.3f', (np.abs(m1-m2), np.sqrt(err1**2 + err2**2)))

    latex_print(f, "")
    latex_command(f, 'pecvelcontribution', '%.3f', np.sqrt(np.mean(a.get_peculiar_velocity_uncertainty()[a.good_mag_mask & a.good_salt_mask]**2)))

    latex_print(f, "")
    latex_host_step(f, 'lssfrsaltxifull', 'lssfr', a.salt_hr, a.good_salt_mask)
    latex_host_step(f, 'gmasssaltxifull', 'gmass', a.salt_hr, a.good_salt_mask)
    latex_host_step(f, 'lssfrsaltxicut', 'lssfr', a.salt_hr, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmasssaltxicut', 'gmass', a.salt_hr, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'lssfrsaltisomapcut', 'lssfr', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmasssaltisomapcut', 'gmass', salt_isomap_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'lssfrrbtlisomapcut', 'lssfr', a.corr_mags, a.good_salt_mask & a.good_mag_mask)
    latex_host_step(f, 'gmassrbtlisomapcut', 'gmass', a.corr_mags, a.good_salt_mask & a.good_mag_mask)
    latex_command(f, 'hostcutsnsnetrain', '%d', np.sum(a.good_mag_mask & a.good_salt_mask & a.host_mask))
    latex_command(f, 'hostcutsnsnefull', '%d', np.sum(a.redshift_color_mask & a.interp_mask & a.good_salt_mask & a.host_mask))

# Outliers

# Tests of dust variation

In [118]:
import extinction
plt.figure()
ref_extinction = extinction.fitzpatrick99(a.wave, 0.5 * 2.8, 2.8)
new_extinction = extinction.fitzpatrick99(a.wave, 0.5 * 3.1, 3.1)
plt.plot(a.wave, ref_extinction - new_extinction)

print("Mean offset:  %.4f" % np.mean(ref_extinction - new_extinction))
print("Sigma offset: %.4f" % np.std(ref_extinction - new_extinction))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean offset:  -0.1395
Sigma offset: 0.0161


In [32]:
import extinction
plt.figure()
ref_extinction = extinction.ccm89(a.wave, 0.5 * 2.8, 2.8)
new_extinction = extinction.ccm89(a.wave, 0.5 * 3.1, 3.1)
plt.plot(a.wave, ref_extinction - new_extinction)

print("Mean offset:  %.4f" % np.mean(ref_extinction - new_extinction))
print("Sigma offset: %.4f" % np.std(ref_extinction - new_extinction))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Mean offset:  -0.1398
Sigma offset: 0.0121
